In [1]:
proj_path='/content/drive/MyDrive/S4 - 16824 VLR/generative-modeling/gan'

### Git Stuff

### Colab stuff

In [ ]:
# one-time data set-up
# #!pip show "clean-fid"
# !cd "$proj_path/$hw_dir" && source "setup_data.sh"
# !ls "/usr/local/lib/python3.7/dist-packages/cleanfid"
# !mkdir "/usr/local/lib/python3.7/dist-packages/cleanfid/stats"
# !cd "$proj_path/$hw_dir/$tasks" && cp cub_clean_custom_na.npz "/usr/local/lib/python3.7/dist-packages/cleanfid/stats/cub_clean_custom_na.npz"

## Notebook

In [1]:
import sys, pathlib
sys.path.append('./gan')
%load_ext autoreload
%autoreload 2

### GAN

In [4]:
from gan.networks import *
from gan.train import *
from gan.q1_3 import compute_discriminator_loss, compute_generator_loss
from networks import Discriminator, Generator

In [ ]:
gen = Generator().cuda().to(memory_format=torch.channels_last)
disc = Discriminator().cuda().to(memory_format=torch.channels_last)
prefix = "data_gan/"
#prefix = "data_wgan_gp/"
os.makedirs(prefix, exist_ok=True)
# TODO 1.3.2: Run this line of code.
train_model(
    gen,
    disc,
    num_iterations=int(3e4),
    batch_size=128,
    prefix=prefix,
    gen_loss_fn=compute_generator_loss,
    disc_loss_fn=compute_discriminator_loss,
    log_period=1000,
    wandb_logging=True
)

data dir exist:True
Epoch after iters:93. D.Loss:1.2965189218521118.G.Loss:0.8051266670227051. LR:[0.0001999628]
Epoch after iters:186. D.Loss:1.3865768909454346.G.Loss:0.9150365591049194. LR:[0.0001999256]
Epoch after iters:279. D.Loss:1.4179582595825195.G.Loss:0.7099700570106506. LR:[0.0001998884]
Epoch after iters:372. D.Loss:1.789136290550232.G.Loss:0.3503444194793701. LR:[0.0001998512]
Epoch after iters:465. D.Loss:1.1580328941345215.G.Loss:1.9012601375579834. LR:[0.000199814]
Epoch after iters:558. D.Loss:1.4779295921325684.G.Loss:1.2967159748077393. LR:[0.0001997768]
Epoch after iters:651. D.Loss:1.1156015396118164.G.Loss:1.2004121541976929. LR:[0.0001997396]
Epoch after iters:744. D.Loss:1.079763412475586.G.Loss:1.2328003644943237. LR:[0.0001997024]
Epoch after iters:837. D.Loss:1.4766446352005005.G.Loss:1.2260570526123047. LR:[0.0001996652]
Epoch after iters:930. D.Loss:1.8002855777740479.G.Loss:1.0721397399902344. LR:[0.000199628]
compute FID of a model with cub-32 statistics

FID model: 100%|███████████████████████████████████| 79/79 [01:37<00:00,  1.24s/it]


Iteration 1000 FID: 337.45226126163635
Epoch after iters:1023. D.Loss:1.4823927879333496.G.Loss:0.299092561006546. LR:[0.0001995908]
Epoch after iters:1116. D.Loss:1.3543026447296143.G.Loss:0.7330020070075989. LR:[0.0001995536]
Epoch after iters:1209. D.Loss:1.4910953044891357.G.Loss:1.0883421897888184. LR:[0.0001995164]
Epoch after iters:1302. D.Loss:1.3836954832077026.G.Loss:0.711723804473877. LR:[0.0001994792]
Epoch after iters:1395. D.Loss:1.381585955619812.G.Loss:0.8841276168823242. LR:[0.000199442]
Epoch after iters:1488. D.Loss:1.3173601627349854.G.Loss:0.8312387466430664. LR:[0.0001994048]
Epoch after iters:1581. D.Loss:1.0486550331115723.G.Loss:0.8983088731765747. LR:[0.0001993676]
Epoch after iters:1674. D.Loss:1.2984663248062134.G.Loss:1.1027106046676636. LR:[0.0001993304]
Epoch after iters:1767. D.Loss:1.4934966564178467.G.Loss:1.0528711080551147. LR:[0.0001992932]
Epoch after iters:1860. D.Loss:1.2324700355529785.G.Loss:0.7679365873336792. LR:[0.00019925600000000003]
Epoch

FID model:  73%|█████████████████████████▋         | 58/79 [01:13<00:19,  1.07it/s]

In [ ]:
print('done')

### VAE

In [ ]:
from vae.model import AEModel, VAEEncoder
from vae.train import vae_loss, main
#from vae.utils import compute_discriminator_loss, compute_generator_loss
#from networks import Discriminator, Generator

In [ ]:
x = torch.randn((2,)+input_shape)
model = AEModel(variational=True, latent_size=128, input_shape = (3, 32, 32))
mu, logvar = model.encoder(x)
print(mu.size(), logvar.size())
print(model.decoder(torch.randn(2, 128)).size())
print(vae_loss(model, x))

In [ ]:
import argparse 
def parse_a2c_arguments():
    # Command-line flags are defined here.
    parser = argparse.ArgumentParser()
    parser.add_argument('--latent_size', dest='latent_size', type=int,
                        default=1024, help="Size of latent space")   # 'LunarLander-v2'
    parser.add_argument('--num_epochs', dest='num_epochs', type=int,
                        default=20, help="Size of latent space")   # 'LunarLander-v2'
    parser.add_argument('--loss_mode', dest='loss_mode', type=str,
                        default='ae', help="Size of latent space")   # 'LunarLander-v2'
    parser.add_argument('--log_dir', dest='log_dir', type=str,
                        default='ae_latent1024', help="directory")
    # ['ae_latent1024','vae_latent1024', 'vae_latent1024_beta_constant0.8','vae_latent1024_beta_linear1']
    parser.add_argument('--beta_mode', dest='beta_mode', type=str,
                        default='constant', help="directorye")   
    # ['constant', 'linear']
    parser.add_argument('--target_beta_val', dest='target_beta_val', type=float,
                        default=0.8, help="final beta")   # 
    # [0.8. 1]
    return parser.parse_known_args()[0]  #parser.parse_args()
args = parse_a2c_arguments()
print(args)

In [ ]:
main(*args.__dict__)